## Hopf Invariants for groups with one relation

This notebook will calculate the Hopf invariants up to a specified depth for a group with a single relation. The formatting for the relation is as follows: 

        rel[i][0] : i^th letter of the relation
        rel[i][1] : exponent of the i^th letter
        
For example: In the cells below we find the Hopf invariants for the group $\langle a, b , c | aba^{-1}c \rangle$ up to weight 4

#### flow idea

    1) initialize lie algebra L[1] (step 1)
    2) initialize vector space V[1]
    3) Find kernel of evaluation of bch(relation) in V[1] (these are Hopf[1])
    4) initialize lie algebra L[2]
    5) initialize vecotr space V[2]
    6) construct cobracket V[2] --> V[1] \tensor V[1]
    7) construct subspace Hopf[1] \tensor Hopf[1] in V[1] tensor V[1]
    8) Find kernel of evaluation of bch(relation) V[2] --> Q
    9) Intersect kernel with preimage of (7) under (6) (these are Hopf[2])
    .
    .
    .

In [11]:
#### ENTER DATA HERE #####

#number of generators (X_0 , X_1 , .... ,X_{numGen-1})
numGen = 3
    
    
#Relation (note indicies start at 0)
rel = [ [0,1] , [1,1] , [0,-1] , [2,1] ]


#depth of Hopf invariants
depth = 4



In [28]:
from sage.algebras.lie_algebras.bch import bch_iterator
Q = VectorSpace(QQ, 1)

In [29]:
#### Initialize Dictionaries #####

L = {}
dimL = {}
V = {}
Vbases = {}
Hopf = {}
HopfStrict = {} #These are the hopf invrts that are not in prior weights
BCHrel= {}
evalBCH = {}
VxV={} #VxV[i] is the vector space for the tensor product of V[i] tensor V[i]
VxVbases= {}
dimHopf ={}
dimHopfStrict={}
HopfxHopf={}
preHopfxHopf ={}
incl ={}



In [30]:
#### Find Hopf[1] #####
print('~~~~~~~~~~~~~~ Weight', 1 , '~~~~~~~~~~~~~~~~~~')
#Initialize L[1] and variables
L[1] = LieAlgebra(QQ , numGen , step = 1)
L[1].inject_variables(verbose=False)
dimL[1] = L[1].dimension()



#Initialize V[1] and Vbases[1]
V[1] = VectorSpace(QQ , dimL[1])

Vbases[1] ={}
k=0
for i in L[1].basis().list():
    Vbases[1][i] = V[1].basis()[k]
    k = k +1 
    

    
#Find BCH relation 
w = rel[0][1]*L[1].basis().list()[rel[0][0]]
for i in range(1,len(rel)):
    b = bch_iterator(w,rel[i][1]*L[1].basis().list()[rel[i][0]])
    w= L[1].zero()
    w= next(b)
    
BCHrel[1]=w

    ########print("BCH relation is :", BCHrel[1])    

#Define eval BCHrel[1]
images = []
for i in L[1].basis().keys():
    images.append(BCHrel[1].coefficient(i)*Q.0 )
    
evalBCH[1] = linear_transformation(V[1], Q , images)
Hopf[1] = evalBCH[1].kernel()
dimHopf[1]= Hopf[1].dimension()
print( '\ndimension =', dimHopf[1] , '\n' )

for i in range(Hopf[1].dimension()):
    print(' Invariant', i+1)
    v = Hopf[1].basis()[i]
    p = L[1].zero()
    for j in range(dimL[1]):
        c = V[1].coordinate_vector(v)[j]
        if c == 0:
            continue
        p = p + c*L[1].basis().list()[j]
    print("    ",p)



##################################
#################################






for d in range(2,depth+1):
    #### Find Hopf[d] #####
    print('~~~~~~~~~~~~~~ Weight', d , '~~~~~~~~~~~~~~~~~~')
    #Initialize L[d] and variables
    L[d] = LieAlgebra(QQ , numGen , step = d)
    L[d].inject_variables(verbose=False )
    dimL[d] = L[d].dimension()
  

    #Initialize V[d] and Vbases[d]
    V[d] = VectorSpace(QQ , dimL[d])

    Vbases[d] ={}
    k=0
    for i in L[d].basis().list():
        Vbases[d][i] = V[d].basis()[k]
        k = k +1 
        
    #Construct inclusion from V[d-1] --> V[d]
    images = [V[d].basis()[k] for k in range(dimL[d-1])]
    incl[d-1] = linear_transformation(V[d-1], V[d], images)
    
    ## Construct Cobracket V[d]--> V[d-1] (X) V[d-1] ##

    #Construct tensor product
    VxV[d-1] = VectorSpace(QQ, dimL[d-1]^2)
    VxVbases[d-1] ={}

    trLb = []
    for i in range(dimL[d-1]):
        trLb.append(L[d].basis().list()[i])

    k = 0
    for j in cartesian_product([trLb, trLb]):
        VxVbases[d-1][j] = VxV[d-1].basis()[k]
        k = k+1

    #define morphism
    tempDic={k : VxV[d-1].zero() for k in L[d].basis().keys()} #these should be (dimL[d])-many elements of VxV[d-1] 

    for j in VxVbases[d-1].keys():
        p = L[d][j[0],j[1]]
        #if bracket is zero then skip to save time
        if p == 0:
            continue
        #if bracket is not zero 
        for k in L[d].basis().keys():
            tempDic[k] = tempDic[k] + p.coefficient(k)*VxVbases[d-1][j]    

    images  = list(tempDic.values())

    cobrack = linear_transformation(V[d], VxV[d-1], images)

    ## Find subspace of VxV[d-1] given by Hopf[d-1] (x) Hopf[d-1]

    HopfxHopf[d-1] = []

    for k in cartesian_product([range(dimHopf[d-1]),range(dimHopf[d-1])]):
        v = Hopf[d-1].basis()[k[0]]
        w = Hopf[d-1].basis()[k[1]]

        t=0
        tens = VxV[d-1].zero()
        for i in cartesian_product([range(dimL[d-1]),range(dimL[d-1])]):
            p=V[d-1].coordinate_vector(v)[i[0]]* V[d-1].coordinate_vector(w)[i[1]]
            tens = tens + p*VxV[d-1].basis()[t]
            t=t+1

        HopfxHopf[d-1].append(tens)

    HopfxHopf[d-1] = VxV[d-1].subspace(HopfxHopf[d-1])


    ## Find the preimage of HopfxHopf[d-1] under cobracket
    preHopfxHopf[d-1] = cobrack.inverse_image(HopfxHopf[d-1])




    #find BCHrel[d]
    l=rel[0][1]*L[d].basis().list()[rel[0][0]]
    for k in range(1, len(rel)):
        b = bch_iterator(l,rel[k][1]*L[d].basis().list()[rel[k][0]])
        l = L[d].zero()
        for i in range(2):
            l = l + next(b)

    BCHrel[d]= l
    
        #######print("BCH relation is :", BCHrel[d])
        
    #Define evalBCH[d]
    images = []

    for i in L[d].basis().keys():
        images.append(BCHrel[d].coefficient(i)*Q.0 )

    evalBCH[d] = linear_transformation(V[d], Q , images)

    #find kernel of eval intersect preHopfxHopf
    resEval = evalBCH[d].restrict_domain(preHopfxHopf[d-1])

    Hopf[d] = resEval.kernel()
    
    
    #Find Hopf invariants in [d] but not [d-1]
    incl[d-1] = incl[d-1].restrict_domain(Hopf[d-1])
    incl[d-1] = incl[d-1].restrict_codomain(Hopf[d])
    
    priorHopf =incl[d-1].image()
    
    Q = Hopf[d]/priorHopf
    
   
    tempList = [Q.lift(v) for v in Q.basis()]
    
    HopfStrict[d] = Hopf[d].subspace(tempList)
    
    dimHopf[d]= Hopf[d].dimension()
    dimHopfStrict[d] = len(tempList)
    #display results
    
    print('\ndimension =', dimHopfStrict[d], '\n')

    for i in range(dimHopfStrict[d]):
        print(' Invariant', i+1)
        v = HopfStrict[d].basis()[i]
        p = L[d].zero()
        for j in range(dimL[d]):
            c = V[d].coordinate_vector(v)[j]
            if c == 0:
                continue
            p = p + c*L[d].basis().list()[j]
        print("    ",p)

~~~~~~~~~~~~~~ Weight 1 ~~~~~~~~~~~~~~~~~~

dimension = 2 

 Invariant 1
     X_1
 Invariant 2
     X_2 - X_3
~~~~~~~~~~~~~~ Weight 2 ~~~~~~~~~~~~~~~~~~

dimension = 1 

 Invariant 1
     X_2 - X_12 + X_13
~~~~~~~~~~~~~~ Weight 3 ~~~~~~~~~~~~~~~~~~

dimension = 2 

 Invariant 1
     X_2 + 4*X_13 - 4*X_112 + 4*X_113
 Invariant 2
     X_23 + X_122 - X_132 + X_133
~~~~~~~~~~~~~~ Weight 4 ~~~~~~~~~~~~~~~~~~

dimension = 3 

 Invariant 1
     X_2 - 4*X_112 - 12*X_113 + 16*X_1112 - 16*X_1113
 Invariant 2
     X_23 + 4*X_123 - 4*X_132 + 4*X_133 + 4*X_1122 - 4*X_1132 + 4*X_1133
 Invariant 3
     X_223 + X_233 - X_1222 + X_1322 - X_1332 + X_1333
